In [1]:
import geopandas as gpd
from shapely.geometry import Polygon
import numpy as np
import os
import rasterio
from rasterio.mask import mask
from shapely import geometry
import math
from rasterio.windows import Window

In [3]:

# points = gpd.read_file('box_se.shp')
# nrows = 3
# ncols = 3
# out_name = "out_rasters/se/box_se_"

in_dir = "data/in"

files = [os.path.join(in_dir,i) for i in os.listdir(in_dir) if i.endswith(".tif")]
#raster_file = "image_99_4BSW4R.tif.tif"
nrows = 3
ncols = 3

out_path = "data/out"

for f in files:
    with rasterio.open(f) as img:
        f = os.path.basename(f)
        xmin, ymin, xmax, ymax = img.bounds

        width = xmax-xmin
        height = ymax-ymin

        wide = width / ncols
        length = height / nrows

        x = xmin
        y = ymin
        polygons = []
        for r in range(nrows):
            if r== nrows-1:
                ystep = length#+(height % nrows)
            else:
                ystep = length

            for c in range(ncols):        
                if (c % ncols) == 0:
                    x = xmin
                if c == ncols-1:
                    xstep = wide#+(width % ncols)
                else:
                    xstep = wide
                polygons.append(Polygon([(x,y), (x+xstep, y), (x+xstep, y+ystep), (x, y+ystep)]))
                x += xstep
            y += ystep


        grid = gpd.GeoDataFrame({'geometry':polygons})
        grid.crs = img.crs

        for i in range(len(grid)):
            bounds = grid.iloc[i]["geometry"]
            out_image, out_transform = mask(img, [bounds], crop=True)
            out_meta = img.meta
            out_meta.update({"driver": "GTiff",
                     "height": out_image.shape[1],
                     "width": out_image.shape[2],
                     "transform": out_transform})
            out_file = os.path.join(out_path,f.split(".")[0]+"_"+str(i)+".tif")
            with rasterio.open(out_file, "w", **out_meta) as dest:
                dest.write(out_image)
        


In [3]:
shape_file = "/home/jovyan/work/notebooks/satellite_data/utils/shapes/2015.shp"

In [22]:
with fiona.open(shape_file) as src_shp:
    shp = src_shp[0]
    s = geometry.shape(shp["geometry"])
    
    xmin, ymin, xmax, ymax = s.bounds

    width = xmax-xmin
    height = ymax-ymin

In [20]:
s.bounds

(29.83169463897843,
 -22.709301697153023,
 29.834255683994552,
 -22.706747206314805)

In [6]:
shp

{'type': 'Feature',
 'id': '0',
 'properties': OrderedDict([('FID', 0)]),
 'geometry': {'type': 'Polygon',
  'coordinates': [[(29.83169463897843, -22.709301697153023),
    (29.83169463897843, -22.706747206314805),
    (29.834255683994552, -22.706747206314805),
    (29.834255683994552, -22.709301697153023),
    (29.83169463897843, -22.709301697153023)]]}}

In [42]:
width

2122649

In [43]:
xstart

2200000

In [6]:
xsize = 100_000
ysize = 100_000
with rasterio.open("/home/jovyan/work/satellite_data/tmp/2015.vrt") as src:
    width = src.width
    height = src.height
    
    nrows = math.ceil(height / ysize)
    ncols = math.ceil(width / xsize)
    
    ystart = 0
    polygons =  []
    for r in range(nrows):
        xstart = 0
        if r < (nrows-1):
            yend = ystart + ysize
        else:
            rest = height - ystart
            yend = ystart + rest
        for c in range(ncols):
            if c < (ncols-1):
                xend = xstart + xsize
            else:
                rest = width - xstart
                xend = xstart + rest
            win = Window.from_slices((ystart,yend),(xstart,xend))
            bounds = src.window_bounds(win)
            box = geometry.box(*bounds)
            polygons.append(box)
            xstart += xsize
        ystart += ysize
        
grid = gpd.GeoDataFrame({'geometry':polygons})
grid.crs = src.crs
grid.to_file("test.shp")

            
        
    

/usr/local/lib/python3.8/dist-packages/geopandas/io/file.py:362: FutureWarning: pandas.Int64Index is deprecated and will be removed from pandas in a future version. Use pandas.Index with the appropriate dtype instead.
  pd.Int64Index,


# Old

In [1]:
import geopandas as gpd
from shapely.geometry import Polygon
import numpy as np
import os

# points = gpd.read_file('box_se.shp')
# nrows = 3
# ncols = 3
# out_name = "out_rasters/se/box_se_"

points = gpd.read_file('box_nw.shp')
nrows = 8
ncols = 8
labels_file = "labels_V5.shp"
out_name = "out_rasters/nw/box_nw_"

xmin, ymin, xmax, ymax = points.total_bounds

width = xmax-xmin
height = ymax-ymin

wide = width / ncols
length = height / nrows

x = xmin
y = ymin
polygons = []
for r in range(nrows):
    if r== nrows-1:
        ystep = length#+(height % nrows)
    else:
        ystep = length
    
    for c in range(ncols):        
        if (c % ncols) == 0:
            x = xmin
        if c == ncols-1:
            xstep = wide#+(width % ncols)
        else:
            xstep = wide
        polygons.append(Polygon([(x,y), (x+xstep, y), (x+xstep, y+ystep), (x, y+ystep)]))
        x += xstep
    y += ystep
        

grid = gpd.GeoDataFrame({'geometry':polygons})
grid.set_crs(epsg=4326, inplace=True)
grid.to_file("grid.shp")

/usr/local/lib/python3.8/dist-packages/geopandas/io/file.py:362: FutureWarning: pandas.Int64Index is deprecated and will be removed from pandas in a future version. Use pandas.Index with the appropriate dtype instead.
  pd.Int64Index,


In [2]:

for i in range(len(grid)):
    bounds = grid.iloc[i]["geometry"].bounds
    gdal_str = "gdal_rasterize -l " + labels_file.split(".")[0] + " -burn 1.0 -tr 2.306434877e-06 2.306434877e-06 -a_nodata 0.0 -te \
                " + str(bounds[0]) + " " + str(bounds[1]) + " "  + str(bounds[2]) + " " + str(bounds[3]) + " \
                " + "-ot Float32 -of GTiff " + labels_file + " " + out_name + str(i) + ".tif"
    os.system(gdal_str)
    
    

0...10...20...30...40...50...60...70...80...90...100 - done.
0...10...20...30...40...50...60...70...80...90...100 - done.
0...10...20...30...40...50...60...70...80...90...100 - done.
0...10...20...30...40...50...60...70...80...90...100 - done.
0...10...20...30...40...50...60...70...80...90...100 - done.
0...10...20...30...40...50...60...70...80...90...100 - done.
0...10...20...30...40...50...60...70...80...90...100 - done.
0...10...20...30...40...50...60...70...80...90...100 - done.
0...10...20...30...40...50...60...70...80...90...100 - done.
0...10...20...30...40...50...60...70...80...90...100 - done.
0...10...20...30...40...50...60...70...80...90...100 - done.
0...10...20...30...40...50...60...70...80...90...100 - done.
0...10...20...30...40...50...60...70...80...90...100 - done.
0...10...20...30...40...50...60...70...80...90...100 - done.
0...10...20...30...40...50...60...70...80...90...100 - done.
0...10...20...30...40...50...60...70...80...90...100 - done.
0...10...20...30...40...